# All

## Set up

In [ ]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

Installing dependencies...
     |████████████████████████████████| 153kB 6.2MB/s 
     |████████████████████████████████| 61kB 4.1MB/s 
     |████████████████████████████████| 3.9MB 7.4MB/s 
     |████████████████████████████████| 3.4MB 40.0MB/s 
     |████████████████████████████████| 1.2MB 49.8MB/s 
     |████████████████████████████████| 2.1MB 45.6MB/s 
     |████████████████████████████████| 245kB 40.5MB/s 
     |████████████████████████████████| 368kB 36.6MB/s 
     |████████████████████████████████| 901kB 44.8MB/s 
     |████████████████████████████████| 3.3MB 47.8MB/s 


In [ ]:
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "v3-8"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU zdetection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)


Setting up GCS access...
Running on TPU: grpc://10.78.126.218:8470
Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
# print(mesh_tensorflow.__version__)

In [ ]:
print(t5.__version__)

0.9.1


In [ ]:
# import gin
# import subprocess
# gin.parse_config_file(
#         'gs://t5-data/pretrained_models/base/operative_config.gin'
#     )


## Register Tasks Medium


In [ ]:
def dumping_dataset(split, shuffle_files = False):
    del shuffle_files
    if split == 'train':
      ds = tf.data.TextLineDataset(
            [
            'gs://t5_training/t5-data/code_data/bug2fix/medium/train.tsv',
            'gs://t5_training/t5-data/code_data/bug2fix/medium/valid.tsv',

            ]
          )
    else:
      ds = tf.data.TextLineDataset(
            [
            ]
          )
    # Split each "<t1>\t<t2>" example into (input), target) tuple.
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                          field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Map each tuple to a {"input": ... "target": ...} dict.
    ds = ds.map(lambda *ex: dict(zip(["input", "target"], ex)))
    return ds

def ner_preprocessor(ds):
  def normalize_text(text):
    return text

  def to_inputs_and_targets(ex):
    """Map {"inputs": ..., "targets": ...}->{"inputs": ner..., "targets": ...}."""
    return {
        "inputs":
             tf.strings.join(
                 ["medium: ", normalize_text(ex["input"])]),
        "targets": normalize_text(ex["target"])
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

print("A few raw validation examples...")
for ex in tfds.as_numpy(dumping_dataset("train").take(5)):
  print(ex)

A few raw validation examples...
{'input': b'public static TYPE_1 init ( java.lang.String name , java.util.Date date )  OPEN_CURLY_TOKEN  TYPE_1 VAR_1 = new TYPE_1 ( ) ; VAR_1 . METHOD_1 ( name ) ; java.util.Calendar VAR_2 = java.util.Calendar.getInstance ( ) ; VAR_2 . METHOD_2 ( date ) ; VAR_1 . METHOD_3 ( VAR_2 ) ; return VAR_1 ;  CLOSE_CURLY_TOKEN ', 'target': b'public static TYPE_1 init ( java.lang.String name , java.util.Date date )  OPEN_CURLY_TOKEN  TYPE_1 VAR_1 = new TYPE_1 ( ) ; VAR_1 . METHOD_1 ( name ) ; java.util.Calendar VAR_2 = null ; if ( date != null )  OPEN_CURLY_TOKEN  VAR_2 = java.util.Calendar.getInstance ( ) ; VAR_2 . METHOD_2 ( date ) ;  CLOSE_CURLY_TOKEN  VAR_1 . METHOD_3 ( VAR_2 ) ; return VAR_1 ;  CLOSE_CURLY_TOKEN '}
{'input': b'public TYPE_1 METHOD_1 ( java.lang.String name )  OPEN_CURLY_TOKEN  if ( name . equals ( STRING_1 ) ) return new TYPE_2 ( STRING_2 , true ) ; if ( name . equals ( STRING_3 ) ) return new TYPE_3 ( STRING_4 , true ) ; if ( name . equals 

In [ ]:
t5.data.TaskRegistry.remove('medium')
t5.data.TaskRegistry.add(
    "medium",
    # Supply a function which returns a tf.data.Dataset.
    dataset_fn=dumping_dataset,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[ner_preprocessor],
    # Lowercase targets before computing metrics.
    # We'll use accuracy as our evaluation metric.
    # output_features=t5.data.Feature(vocabulary=t5.data.SentencePieceVocabulary(vocab)),

    # metric_fns=[t5.evaluation.metrics.accuracy, 
    #            t5.evaluation.metrics.sequence_accuracy, 
    #             ],
    # output_features=t5.data.Feature(vocabulary=t5.data.SentencePieceVocabulary(vocab))
)

## Register Tasks small


In [ ]:
def dumping_dataset(split, shuffle_files = False):
    del shuffle_files
    if split == 'train':
      ds = tf.data.TextLineDataset(
            [
            'gs://t5_training/t5-data/code_data/bug2fix/small/train.tsv',
            'gs://t5_training/t5-data/code_data/bug2fix/small/valid.tsv',

            ]
          )
    else:
      ds = tf.data.TextLineDataset(
            [
            ]
          )
    # Split each "<t1>\t<t2>" example into (input), target) tuple.
    ds = ds.map(
        functools.partial(tf.io.decode_csv, record_defaults=["", ""],
                          field_delim="\t", use_quote_delim=False),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # Map each tuple to a {"input": ... "target": ...} dict.
    ds = ds.map(lambda *ex: dict(zip(["input", "target"], ex)))
    return ds

def ner_preprocessor(ds):
  def normalize_text(text):
    return text

  def to_inputs_and_targets(ex):
    """Map {"inputs": ..., "targets": ...}->{"inputs": ner..., "targets": ...}."""
    return {
        "inputs":
             tf.strings.join(
                 ["small: ", normalize_text(ex["input"])]),
        "targets": normalize_text(ex["target"])
    }
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

print("A few raw validation examples...")
for ex in tfds.as_numpy(dumping_dataset("train").take(5)):
  print(ex)

A few raw validation examples...
{'input': b'public java.lang.String METHOD_1 ( )  OPEN_CURLY_TOKEN  return new TYPE_1 ( STRING_1 ) . format ( VAR_1  OPEN_SQUARE_TOKEN  ( ( VAR_1 . length ) - 1 )  CLOSE_SQUARE_TOKEN  . getTime ( ) ) ;  CLOSE_CURLY_TOKEN ', 'target': b'public java.lang.String METHOD_1 ( )  OPEN_CURLY_TOKEN  return new TYPE_1 ( STRING_1 ) . format ( VAR_1  OPEN_SQUARE_TOKEN  ( ( type ) - 1 )  CLOSE_SQUARE_TOKEN  . getTime ( ) ) ;  CLOSE_CURLY_TOKEN '}
{'input': b'public boolean METHOD_1 ( java.lang.String name )  OPEN_CURLY_TOKEN  TYPE_1 VAR_1 = TYPE_1 . METHOD_2 ( VAR_2 ) ; return ( ! ( METHOD_3 ( name ) ) ) && ( VAR_1 . contains ( name ) ) ;  CLOSE_CURLY_TOKEN ', 'target': b'public boolean METHOD_1 ( java.lang.String name )  OPEN_CURLY_TOKEN  return ( ! ( METHOD_3 ( name ) ) ) && ( TYPE_1 . METHOD_2 ( VAR_2 ) . contains ( name ) ) ;  CLOSE_CURLY_TOKEN '}
{'input': b'public char METHOD_1 ( java.lang.String VAR_1 , java.lang.String name )  OPEN_CURLY_TOKEN  return null ;

In [ ]:
t5.data.TaskRegistry.remove('small')
t5.data.TaskRegistry.add(
    "small",
    # Supply a function which returns a tf.data.Dataset.
    dataset_fn=dumping_dataset,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[ner_preprocessor],
    # Lowercase targets before computing metrics.
    # We'll use accuracy as our evaluation metric.
    # output_features=t5.data.Feature(vocabulary=t5.data.SentencePieceVocabulary(vocab)),

    # metric_fns=[t5.evaluation.metrics.accuracy, 
    #            t5.evaluation.metrics.sequence_accuracy, 
    #             ],
    # output_features=t5.data.Feature(vocabulary=t5.data.SentencePieceVocabulary(vocab))
)

## Mixtures

In [ ]:
t5.data.MixtureRegistry.remove("all_mix")
t5.data.MixtureRegistry.add(
    "all_mix",
    [
     'medium',
     'small'
     ],
     default_rate=1.0
)

## Define Model

In [ ]:
# !gsutil -m rm -r {MODEL_DIR}

In [ ]:
# Using pretrained_models from wiki + books
MODEL_SIZE = "base"
BASE_PRETRAINED_DIR = "gs://t5-data/pretrained_models"
# BASE_PRETRAINED_DIR = "gs://t5_training/models/code/code_uni_v1"


# BASE_PRETRAINED_DIR = "gs://t5_training/models/export_models/code/code_all_v1/"
# BASE_PRETRAINED_DIR = "gs://t5_training/models/export_models/code/code_only_code_v1/"
# BASE_PRETRAINED_DIR = "gs://t5_training/models/export_models/code/code_all_codesearchnet_v1/"

PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
# MODEL_DIR = "gs://t5_training/models/bio/re_v2"
MODEL_DIR = "gs://t5_training/models/code/bug2fix_base_v2/"
MODEL_DIR = os.path.join(MODEL_DIR, MODEL_SIZE)


# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 512, "targets": 512},
    learning_rate_schedule=0.001,
    save_checkpoints_steps=1000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)


## Finetune

In [ ]:
FINETUNE_STEPS = 45000

model.finetune(
    mixture_or_task_name="all_mix",
    pretrained_model_dir=PRETRAINED_DIR,
    finetune_steps=FINETUNE_STEPS
)

INFO:root:system_path_file_exists:gs://t5-data/pretrained_models/base/operative_config.gin
ERROR:root:Path not found: gs://t5-data/pretrained_models/base/operative_config.gin
INFO:root:Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).


INFO:tensorflow:Using config: {'_model_dir': 'gs://t5_training/models/code/bug2fix_base_v2/base', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.58.40.26:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.58.40.26:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.58.40.26:8470', '_evaluation_master': 'grpc://10.58.40.26:847

/usr/local/lib/python3.7/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[4, 2] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[2] physical_shape=[1, 1, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1)]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=4, model=2] LayoutRules{('vocab', 'model'), ('batch', 'batch'), ('heads', 'model'),

## Predict

In [ ]:
tasks = [
         ['bug2fix', 'small'],
         ['bug2fix', 'medium']
         ]
output_dir = "bug2fix_base_v2"
test_file = 'test'
%cd /content/

/content


In [ ]:
for task in tasks:
  !mkdir {task[1]}
  !gsutil cp gs://t5_training/t5-data/code_data/{task[0]}/{task[1]}/{test_file}.tsv {task[1]}/
  with open(f'{task[1]}/{test_file}.tsv', 'r') as file:
    with open(f'{task[1]}/predict_input.tsv', 'w') as predict_input:
      with open(f'{task[1]}/actual_output.tsv', 'w') as actual_output:
        for line in file:
          line = line.strip().split('\t')
          input = line[0].strip()
          
          actual = line[1].strip()
          actual = ' '.join(actual.split())
          actual = actual.strip() \
                .replace(' SMALLER_TOKEN ', ' < ') \
                .replace(' GREATER_TOKEN ', ' > ')\
                .replace(' OPEN_SQUARE_TOKEN ', ' [ ')\
                .replace(' CLOSE_SQUARE_TOKEN ', ' ] ')\
                .replace(' OPEN_CURLY_TOKEN ', ' { ')\
                .replace(' CLOSE_CURLY_TOKEN', ' } ')\
                .replace(' CLOSE_CURLY_TOKEN ', ' } ')\
                .replace(' EXPONENTIAL_TOKEN ', ' ^ ')\
                .replace(' SHARP_TOKEN ', ' # ')\
                .replace(' DOLLAR_TOKEN ', ' $ ')\
                .replace(' UNK_TOKEN ', ' ` ') \
                .replace(' NEW_LINE ', ' \\n ') \
                .replace(' INDENT ', ' \\t ')


          # actual = actual.strip() \
          #       .replace(' SMALLER_TOKEN ', 'SMALLER_TOKEN') \
          #       .replace('  GREATER_TOKEN ', 'GREATER_TOKEN')\
          #       .replace(' OPEN_SQUARE_TOKEN ', 'OPEN_SQUARE_TOKEN')\
          #       .replace(' CLOSE_SQUARE_TOKEN ', 'CLOSE_SQUARE_TOKEN')\
          #       .replace(' OPEN_CURLY_TOKEN ', 'OPEN_CURLY_TOKEN')\
          #       .replace(' CLOSE_CURLY_TOKEN ', 'CLOSE_CURLY_TOKEN')\
          #       .replace(' EXPONENTIAL_TOKEN ', 'EXPONENTIAL_TOKEN')\
          #       .replace(' SHARP_TOKEN ', 'SHARP_TOKEN')\
          #       .replace(' DOLLAR_TOKEN ', 'DOLLAR_TOKEN')\
          #       .replace(' UNK_TOKEN ', 'UNK_TOKEN') \
          #       .replace(' NEW_LINE ', 'NEW_LINE') \
          #       .replace(' INDENT ', 'INDENT')
          # break


          predict_input.write(f'{task[1]}: {input}\n')
          actual_output.write(f'{actual}\n')
          # print(actual)
          # break

mkdir: cannot create directory ‘small’: File exists
Copying gs://t5_training/t5-data/code_data/bug2fix/small/test.tsv...
/ [1 files][  2.1 MiB/  2.1 MiB]                                                
Operation completed over 1 objects/2.1 MiB.                                      
Copying gs://t5_training/t5-data/code_data/bug2fix/medium/test.tsv...
/ [1 files][  5.2 MiB/  5.2 MiB]                                                
Operation completed over 1 objects/5.2 MiB.                                      


In [ ]:
import tensorflow.compat.v1 as tf
# question_1 = "Emerin is a nuclear membrane protein which is missing or defective in Emery-Dreifuss muscular dystrophy (EDMD). It is one member of a family of lamina-associated proteins which includes LAP1, LAP2 and lamin B receptor (LBR). A panel of 16 monoclonal antibodies (mAbs) has been mapped to six specific sites throughout the emerin molecule using phage-displayed peptide libraries and has been used to localize emerin in human and rabbit heart. Several mAbs against different emerin epitopes did not recognize intercalated discs in the heart, though they recognized cardiomyocyte nuclei strongly, both at the rim and in intranuclear spots or channels. A polyclonal rabbit antiserum against emerin did recognize both nuclear membrane and intercalated discs but, after affinity purification against a pure-emerin band on a western blot, it stained only the nuclear membrane. These results would not be expected if immunostaining at intercalated discs were due to a product of the emerin gene and, therefore, cast some doubt upon the hypothesis that cardiac defects in EDMD are caused by absence of emerin from intercalated discs. Although emerin was abundant in the membranes of cardiomyocyte nuclei, it was absent from many non-myocyte cells in the heart. This distribution of emerin was similar to that of lamin A, a candidate gene for an autosomal form of EDMD. In contrast, lamin B1 was absent from cardiomyocyte nuclei, showing that lamin B1 is not essential for localization of emerin to the nuclear lamina. Lamin B1 is also almost completely absent from skeletal muscle nuclei. In EDMD, the additional absence of lamin B1 from heart and skeletal muscle nuclei which already lack emerin may offer an alternative explanation of why these tissues are particularly affected.." 
# question_2 = "Molecular analysis of the APC gene in 205 families: extended genotype-phenotype correlations in FAP and evidence for the role of APC amino acid changes in colorectal cancer predisposition." 
# question_3 = "Who are the 4 members of The Beatles?" 
# question_4 = "How many teeth do humans have?"

# questions = [question_2]


for task in tasks:
  dir = task[0]
  input_file = f'{task[1]}/predict_input.tsv'
  output_file = f'{task[1]}/predict_output.tsv'


  # Write out the supplied questions to text files.
  # predict_inputs_path = os.path.join('gs://t5_training/t5-data/code_data', dir, input_file)
  # predict_outputs_path = os.path.join('gs://t5_training/t5-data/code_data', dir, output_dir , MODEL_SIZE, output_file)

  predict_inputs_path = input_file
  predict_outputs_path = output_file

  # Manually apply preprocessing by prepending "triviaqa question:".
  print(predict_inputs_path)
  print(predict_outputs_path)
  # Ignore any logging so that we only see the model's answers to the questions.
  with tf_verbosity_level('ERROR'):
    model.batch_size = 8  # Min size for small model on v2-8 with parallelism 1.
    model.predict(
        input_file=predict_inputs_path,
        output_file=predict_outputs_path,
        checkpoint_steps=-1,
        # Select the most probable output token at each step.
        # vocabulary=t5.data.SentencePieceVocabulary(vocab)
        temperature=0,
    )

  # The output filename will have the checkpoint appended so we glob to get 
  # the latest.
  prediction_files = sorted(tf.io.gfile.glob(predict_outputs_path + "*"))
  print("Predicted task : " + dir)
  print("\nPredictions using checkpoint %s:\n" % prediction_files[-1].split("-")[-1])
  # with tf.io.gfile.GFile(prediction_files[-1]) as f:
  #   for q, a in zip(questions, f):
  #     if q:
  #       print("Q: " + q)
  #       print("A: " + a)
  #       print()

small/predict_input.tsv
small/predict_output.tsv


INFO:root:system_path_file_exists:gs://t5_training/models/code/bug2fix_base_v2/base/operative_config.gin
ERROR:root:Path not found: gs://t5_training/models/code/bug2fix_base_v2/base/operative_config.gin


Predicted task : bug2fix

Predictions using checkpoint 1044900:

medium/predict_input.tsv
medium/predict_output.tsv


INFO:root:system_path_file_exists:gs://t5_training/models/code/bug2fix_base_v2/base/operative_config.gin
ERROR:root:Path not found: gs://t5_training/models/code/bug2fix_base_v2/base/operative_config.gin


Predicted task : bug2fix

Predictions using checkpoint 1044900:



## Scoring

In [ ]:
!git clone https://github.com/microsoft/CodeXGLUE.git
%cd /content/CodeXGLUE/Code-Code/code-refinement

Cloning into 'CodeXGLUE'...
remote: Enumerating objects: 2604, done.
remote: Counting objects: 100% (261/261), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 2604 (delta 160), reused 67 (delta 41), pack-reused 2343
Receiving objects: 100% (2604/2604), 200.43 MiB | 21.63 MiB/s, done.
Resolving deltas: 100% (1230/1230), done.
Checking out files: 100% (385/385), done.
/content/CodeXGLUE/Code-Code/code-refinement


In [ ]:
tasks = [
         ['bug2fix', 'small'],
         ['bug2fix', 'medium']
         ]
# output_dir = "defect_detection_code_all_codesearchnet_v1"
test_file = 'test'
checkpoint = '1044900'

In [ ]:
for task in tasks:
  print(task[0], task[1])
  
  with open(f'/content/{task[1]}/predict_output.tsv-{checkpoint}') as file:
    with open(f'/content/{task[1]}/predict_output.tsv', 'w') as out_file:
      for line in file:
        line = line.strip() \
                .replace('SMALLER_TOKEN', '<') \
                .replace('GREATER_TOKEN', '>')\
                .replace('OPEN_SQUARE_TOKEN', '[')\
                .replace('CLOSE_SQUARE_TOKEN', ']')\
                .replace('OPEN_CURLY_TOKEN', '{')\
                .replace('CLOSE_CURLY_TOKEN', '}')\
                .replace('EXPONENTIAL_TOKEN', '^')\
                .replace('SHARP_TOKEN', '#')\
                .replace('DOLLAR_TOKEN', '$')\
                .replace('UNK_TOKEN', '`') \
                .replace('NEW_LINE', '\\n') \
                .replace('INDENT', '\\t')
        out_file.write(f'{line}\n')
  
  !python evaluator/evaluator.py -ref /content/{task[1]}/actual_output.tsv -pre /content/{task[1]}/predict_output.tsv


bug2fix small
BLEU: 77.43 ; Acc: 16.04
bug2fix medium
BLEU: 88.28 ; Acc: 4.11


In [ ]:
!pip install tree_sitter==0.2.0
%cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU

for task in tasks:
  print(task[0], task[1])
  !python calc_code_bleu.py --refs /content/{task[1]}/actual_output.tsv --hyp /content/{task[1]}/predict_output.tsv --lang java

/content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU
bug2fix small
ngram match: 0.7742618221241298, weighted ngram match: 0.7872707245755032, syntax_match: 0.7397810520897484, dataflow_match: 0.7869320499225059
CodeBLEU score:  0.7720614121779718
bug2fix medium
ngram match: 0.8827828757167933, weighted ngram match: 0.8833559860946272, syntax_match: 0.8254292584355915, dataflow_match: 0.8331314870234509
CodeBLEU score:  0.8561749018176157
